In [3]:
import pandas as pd
import numpy as np
from os.path import join

In [409]:
DATA_FOLDER = '../raw_data'
IPEDS_FOLDER = join(DATA_FOLDER, 'IPEDS_Dataset_Encoded')

# Enrollent
EFFY_FILE = join(IPEDS_FOLDER, '12-Month Enrollment/EFFY_2015-2021_data.csv')
EFIA_FILE = join(IPEDS_FOLDER, '12-Month Enrollment/EFIA_2015-2021_data.csv')

# Libraries
AL_FILE = join(IPEDS_FOLDER, 'Academic Libraries/AL_2015-2020_data.csv')

# Admissions
ADM_FILE = join(IPEDS_FOLDER, 'Admission and Test Scores/ADM_2015-2021_data.csv')

# Completions
COMPLETION_A_FILE = join(IPEDS_FOLDER, 'Completions/C_A_2015-2021_data.csv')
COMPLETION_B_FILE = join(IPEDS_FOLDER, 'Completions/C_B_2015-2021_data.csv')
COMPLETION_C_FILE = join(IPEDS_FOLDER, 'Completions/C_C_2015-2021_data.csv')
COMPLETION_CDEP_FILE = join(IPEDS_FOLDER, 'Completions/CDEP_2015-2021_data.csv')

# Employment
EMP_FILE = join(IPEDS_FOLDER, 'Employees by Assigned Position/EAP_2015-2020_data.csv')

# Fall Enrollment
EFA_FILE = join(IPEDS_FOLDER, 'Fall Enrollment/EFA_2015-2020_data.csv')
EFA_DIST_FILE = join(IPEDS_FOLDER, 'Fall Enrollment/EFA_DIST_2015-2020_data.csv')
EFB_FILE = join(IPEDS_FOLDER, 'Fall Enrollment/EFB_2015-2020_data.csv')
EFC_FILE = join(IPEDS_FOLDER, 'Fall Enrollment/EFC_2015-2020_data.csv')
EFD_FILE = join(IPEDS_FOLDER, 'Fall Enrollment/EFD_2015-2020_data.csv')

# Graduation Rates
GRAD_RATE_FILE = join(IPEDS_FOLDER, 'Graduation Rates/GR_2015-2021_data.csv')
GRAT_RATE_ABN_FILE = join(IPEDS_FOLDER, 'Graduation Rates/GR_L2_2015-2021_data.csv')

# Institutional Characteristics
DIRECTORY_FILE = join(IPEDS_FOLDER, 'Institutional Characteristics/HD_2015-2021_data.csv')
IC_FILE = join(IPEDS_FOLDER, 'Institutional Characteristics/IC_2015-2021_data.csv')
IC_ACADEMIC_FILE = join(IPEDS_FOLDER, 'Institutional Characteristics/IC_AY_2015-2021_data.csv')
IC_PROGRAM_FILE = join(IPEDS_FOLDER, 'Institutional Characteristics/IC_PY_2015-2021_data.csv')

# Institutional Finances
FINANCE_PUB_FILE = join(IPEDS_FOLDER, 'Institutional Finances/F_F1A_1415-1920_data.csv')
FINANCE_PRINPF_FILE = join(IPEDS_FOLDER, 'Institutional Finances/F_F2_1415-1920_data.csv')
FINANCE_PRI_FILE = join(IPEDS_FOLDER, 'Institutional Finances/F_F3_1415-1920_data.csv')

# Outcome Measures
OUTCOME_FILE = join(IPEDS_FOLDER, 'Outcome Measures/OM_2015-2021_data.csv')

# Student Financial Aid
FINAID_FILE = join(IPEDS_FOLDER, 'Student Financial Aid/SFA_1415-2021_data.csv')
FINIAD_VET = join(IPEDS_FOLDER, 'Student Financial Aid/SFAV_1415-2021_data.csv')


In [410]:
employees_df = pd.read_csv(EMP_FILE)
faculty_employees_df = employees_df[employees_df['eapcat'] == 'Instructional staff, With faculty status'][['unitid', 'eapft', 'year']].rename(columns={'eapft':'Total Faculty'}).sort_values('unitid')
grad_employees_df = employees_df[employees_df['eapcat'] == 'Graduate Assistants'][['unitid', 'eaptyp', 'year']].rename(columns={'eaptyp':'Total Grad Students'}).sort_values('unitid')
employees_df = faculty_employees_df.merge(grad_employees_df, on=['unitid','year'], how='left')
employees_df['Total Employees Weighted'] = employees_df['Total Faculty']*2.5 + employees_df['Total Grad Students']

In [411]:
finance_public_df = pd.read_csv(FINANCE_PUB_FILE)
finance_private_nonprofit_df = pd.read_csv(FINANCE_PRINPF_FILE)

FPUCOLS = ['f1c011', 'f1c021', 'f1c031', 'f1c051', 'f1c061', 'f1c071', 'f1c081', 'f1c101', 'f1c111', 'f1c121', 'f1c131']
current = finance_public_df[FPUCOLS[0]].fillna(0)
for c in FPUCOLS[1:]:
    current += finance_public_df[c].fillna(0)
finance_public_df['Current Year Total Cost'] = current
fpucolsmapper = {'f1c012':'Total Instruction Salary', 'f1c011': 'Total Instruction', 'f1c021': 'Total Research', 'f1c031': 'Total Public Service', 'f1c051': 'Total Academic Support', 'f1c061': 'Total Student Services', 'f1c071': 'Total Institutional Support', 'f1c081': 'Total Operation Maintenance', 'f1c101': 'Total Scholarships', 'f1c111': 'Total Auxillary Enterprises', 'f1c121': 'Total Hospital Services', 'f1c131': 'Total Independent Operations'}
fpu = finance_public_df[['unitid','year', 'f1c012', 'Current Year Total Cost'] + FPUCOLS].rename(columns=fpucolsmapper)

fprncols = ['f2e011', 'f2e021', 'f2e031', 'f2e041', 'f2e051', 'f2e061', 'f2e071', 'f2e081', 'f2e091', 'f2e101', 'f2e111']
finance_private_nonprofit_df['Current Year Total Cost'] = finance_private_nonprofit_df['f2e131']
fprncolsmapper = {'f2e012':'Total Instruction Salary', 'f2e011': 'Total Instruction', 'f2e021': 'Total Research', 'f2e031': 'Total Public Service', 'f2e041': 'Total Academic Support', 'f2e051': 'Total Student Services', 'f2e061': 'Total Institutional Support', 'f2e111': 'Total Operation Maintenance', 'f2e081': 'Total Scholarships', 'f2e071': 'Total Auxillary Enterprises', 'f2e091': 'Total Hospital Services', 'f2e101': 'Total Independent Operations'}
fprn = finance_private_nonprofit_df[['unitid', 'year', 'f2e012', 'Current Year Total Cost'] + fprncols].rename(columns=fprncolsmapper)

finances_df = pd.concat([fprn, fpu]).fillna(0)

yearmapper = {1415:2015, 1516:2016, 1617:2017, 1718:2018, 1819:2019, 1920:2020}
finances_df['year'] = finances_df['year'].apply(yearmapper.get)

/Users/peterzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (177) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [412]:
effy_df = pd.read_csv(EFFY_FILE)
undergrad_count_df = effy_df[effy_df['effylev'] == 'Undergraduate'][['unitid', 'year', 'efytotlt']].rename(columns={'efytotlt':'Total Undergrads'})

/Users/peterzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [413]:
admin_df = pd.read_csv(ADM_FILE)
admin_cols = ['unitid','year','actcm75', 'admit_rate'] # get act
admin_df['admit_rate'] = admin_df['admssn'] / admin_df['applcn']
admin_df = admin_df[admin_cols]

In [437]:
finaid_df = pd.read_csv(FINAID_FILE)
finaid_df['year'] = finaid_df['year'].apply(yearmapper.get)
finaid_cols = ['unitid','year','upgrntp']
finaid_df = finaid_df[finaid_cols]

,unitid,year,upgrntp
0,100654,2015.0,72
1,100663,2015.0,35
2,100690,2015.0,75
3,100706,2015.0,32
4,100724,2015.0,76


### Graduate Salary

In [415]:
# Payscale
PAY_FILE = 'PayScale Salary Data 2021.csv'

In [416]:
pay_df = pd.read_csv(join(DATA_FOLDER, PAY_FILE))
directory_df = pd.read_csv(DIRECTORY_FILE)

/Users/peterzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (13,23,48,49,50,51,52,53,54,70,71,74,75,76,77,78,79,80,81,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [417]:
pay_df_merged = pay_df.merge(directory_df[directory_df['year'] == 2021], left_on='School Name', right_on='instnm', how = 'left')
pay_df_merged = pay_df_merged.drop_duplicates()

In [418]:
pay_df_cols = ['unitid','Early Career Pay', 'Mid-Career Pay','% High Meaning', '% STEM Degrees']
pay_df_merged = pay_df_merged[pay_df_cols]
pay_df_merged.dropna(axis=0, subset=['unitid'], inplace=True)
pay_df_merged['unitid'] = pay_df_merged['unitid'].astype(int)

### Cost of Living

In [419]:
FIPS_FILE = join(DATA_FOLDER, 'fips2county.csv')
LIVING_WAGE_FILE = join(DATA_FOLDER, 'living-wage.json')
fips_df = pd.read_csv(FIPS_FILE, encoding='latin-1')
living_wage_df = pd.read_json(LIVING_WAGE_FILE).transpose().reset_index()

In [420]:
fips_df['county_name'] = fips_df['name'] + ', ' + fips_df['state']
county2fips = dict(zip(fips_df['county_name'], fips_df['fips']))
directory_df['fips'] = directory_df['countycd'].apply(county2fips.get)

In [421]:
cola_cols = ['unitid', 'living_wage']
cola_df = directory_df.merge(living_wage_df, left_on = 'fips', right_on = 'index', how = 'left')
cola_df['living_wage'] = cola_df['livingWage-1-adult'].str[1:].astype(float)
cola_df = cola_df[cola_cols].drop_duplicates().dropna()

## Merging

In [422]:
directory_df['is_medical'] = (directory_df['medical'] == 'Yes').astype(int)

In [438]:
combined_df = directory_df[['unitid', 'instnm', 'year', 'is_medical']]
UNIT_YEAR_DF = [employees_df, finances_df, undergrad_count_df, admin_df, finaid_df]
UNIT_DF = [cola_df, pay_df_merged] # for every 
for df in UNIT_YEAR_DF:
    print(len(combined_df))
    combined_df = combined_df.merge(df, on=['unitid','year'], how = 'inner')
for df in UNIT_DF:
    combined_df = combined_df.merge(df, on='unitid', how= 'inner')

48466
22203
18466
17229
9661


In [439]:
combined_df['Percent Instruction Salary'] = combined_df['Total Instruction Salary'] / combined_df['Current Year Total Cost']
combined_df['Salary Index'] = combined_df['Total Instruction Salary'] / combined_df['Total Employees Weighted'] / combined_df['living_wage'] * 10
combined_df['Student/Faculty Ratio'] = combined_df['Total Undergrads'] / combined_df['Total Faculty']
combined_df['Salary Score'] = combined_df['Percent Instruction Salary'] * combined_df['Salary Index'] / combined_df['Student/Faculty Ratio']
combined_df = combined_df[combined_df['Total Grad Students'] >= 100][combined_df['year'] == 2020].sort_values('Salary Score', ascending=False)
combined_df.head()

<ipython-input-439-7225cb9ee3bb>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  combined_df = combined_df[combined_df['Total Grad Students'] >= 100][combined_df['year'] == 2020].sort_values('Salary Score', ascending=False)


,unitid,instnm,year,is_medical,Total Faculty,Total Grad Students,Total Employees Weighted,Total Instruction Salary,Current Year Total Cost,Total Instruction,...,upgrntp,living_wage,Early Career Pay,Mid-Career Pay,% High Meaning,% STEM Degrees,Percent Instruction Salary,Salary Index,Student/Faculty Ratio,Salary Score
4207,190150,Columbia University in the City of New York,2020,1,4327.0,2493.0,13310.5,1739837000,5.037252e+09,3126101000,...,23,21.77,"$78,200","$138,200",45%,33%,0.345394,60042.084817,2.008089,10327.322662
3729,179867,Washington University in St Louis,2020,1,1717.0,1149.0,5441.5,1091457583,3.560028e+09,2300416000,...,13,14.23,"$70,000","$126,400",43%,31%,0.306587,140955.922732,5.188701,8328.717161
1526,144050,University of Chicago,2020,1,2376.0,810.0,6750.0,971540370,4.068781e+09,1459805561,...,11,16.32,"$70,700","$131,700",38%,25%,0.238779,88193.570261,3.452862,6098.938060
730,123961,University of Southern California,2020,1,2403.0,1669.0,7676.5,1356804000,5.529573e+09,2457306631,...,20,19.35,"$70,400","$133,300",47%,23%,0.245372,91342.499534,9.054099,2475.444365
7530,243744,Stanford University,2020,1,2106.0,3060.0,8325.0,1098403000,6.065925e+09,2081741000,...,18,27.29,"$87,100","$156,500",55%,50%,0.181078,48347.490033,4.018993,2178.318194


In [440]:
OUTPUT_FOLDER = '../data'
OUTPUT_FILE = join(OUTPUT_FOLDER, 'train.csv')
combined_df.to_csv(OUTPUT_FILE)

In [446]:
combined_df

,unitid,instnm,year,is_medical,Total Faculty,Total Grad Students,Total Employees Weighted,Total Instruction Salary,Current Year Total Cost,Total Instruction,...,upgrntp,living_wage,Early Career Pay,Mid-Career Pay,% High Meaning,% STEM Degrees,Percent Instruction Salary,Salary Index,Student/Faculty Ratio,Salary Score
4207,190150,Columbia University in the City of New York,2020,1,4327.0,2493.0,13310.5,1739837000,5.037252e+09,3126101000,...,23,21.77,"$78,200","$138,200",45%,33%,0.345394,60042.084817,2.008089,10327.322662
3729,179867,Washington University in St Louis,2020,1,1717.0,1149.0,5441.5,1091457583,3.560028e+09,2300416000,...,13,14.23,"$70,000","$126,400",43%,31%,0.306587,140955.922732,5.188701,8328.717161
1526,144050,University of Chicago,2020,1,2376.0,810.0,6750.0,971540370,4.068781e+09,1459805561,...,11,16.32,"$70,700","$131,700",38%,25%,0.238779,88193.570261,3.452862,6098.938060
730,123961,University of Southern California,2020,1,2403.0,1669.0,7676.5,1356804000,5.529573e+09,2457306631,...,20,19.35,"$70,400","$133,300",47%,23%,0.245372,91342.499534,9.054099,2475.444365
7530,243744,Stanford University,2020,1,2106.0,3060.0,8325.0,1098403000,6.065925e+09,2081741000,...,18,27.29,"$87,100","$156,500",55%,50%,0.181078,48347.490033,4.018993,2178.318194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6758,227216,University of North Texas,2020,0,1127.0,1627.0,4444.5,128416920,6.951984e+08,213200266,...,36,15.21,"$54,200","$98,000",47%,15%,0.184720,18996.348344,32.531500,107.864752
148,102614,University of Alaska Fairbanks,2020,0,289.0,281.0,1003.5,44852608,3.741030e+08,85442514,...,23,14.79,"$61,600","$106,000",52%,20%,0.119894,30220.535091,35.363322,102.457937
2447,159647,Louisiana Tech University,2020,0,364.0,271.0,1181.0,31241339,1.897795e+08,60050712,...,23,14.12,"$57,500","$103,900",55%,25%,0.164619,18734.626751,30.956044,99.627654
112,102094,University of South Alabama,2020,1,801.0,253.0,2255.5,56355000,8.752580e+08,134791000,...,39,14.37,"$51,800","$86,500",57%,16%,0.064387,17387.328308,13.163546,85.046479
